## Imports and constants

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio 

In [2]:
RESULTS_FILE = 'aoa_all.csv'
ALL_PROPENSITIES = {'original': "Original", 'bb-item-user': "BB Item-User", 'bb-item': 'BB Item'}
ALL_METHODS = { 'bpr': 'BPR',
            'ubpr' : 'UBPR',
            'dubpr': 'DUBPR',
            'relmf' : 'RelMF',
            'dumf': 'DUMF',
            'expomf' : 'ExpoMF',
            'ip' : 'ItemPop',
            'wmf' : 'WMF',
            'ngcf-b': 'NGCF-BPR',
            'ngcf-u': 'NGCF-UBPR',
        }

## Define functions for loading and plotting results

In [3]:
def load_df(results_path, data, propensity, method, results_file):
    try :
        df = pd.read_csv(f"{results_path}/{data}/{propensity}/{method}/results/{results_file}")
    except:
        print(f"Error loading {results_path}/{data}/{propensity}/{method}/results/{results_file}")
        print(f"It is required to run the models for {data} dataset with {propensity} propensity and {method} method")
        raise Exception("Missing results")
    df = df.rename(columns={'Unnamed: 0': 'Metric@K'})
    metrics =  df['Metric@K'].to_list()
    df = df.iloc[:, 1:].transpose()
    df.columns = metrics
    df.reset_index(inplace=True, drop=True)
    df['Method'] = method
    df = df.melt(id_vars=['Method'], var_name='Metric@K', value_name='Value')
    df['Propensity'] = ALL_PROPENSITIES[propensity]
    df['Metric'] = df['Metric@K'].apply(lambda x: x.split('@')[0])
    df['K'] = df['Metric@K'].apply(lambda x: x.split('@')[1])
    return df

In [4]:
def load_results(results_path, data, propensities, methods):
    """Load all results for a given dataset"""
    dfs = []
    for propensity in propensities:
        for method in methods:
            df = load_df(results_path, data, propensity, method, RESULTS_FILE)
            dfs.append(df)
    return pd.concat(dfs)

def plot_metric(df, data, metric, methods=None, propensities=None):
    """Plot a metric for given methods and propensities"""
    if methods:
        df = df[df['Method'].isin(methods)]
    if propensities:
        df = df[df['Propensity'].isin(propensities)]
    df = df[df['Metric'] == metric]
    if len(propensities) == 1:
       fig = px.box(df, x="Metric@K", y="Value", color="Method")
    else:
       fig = px.box(df, x="Metric@K", y="Value", color="Method", facet_col='Propensity')
    fig.update_traces(quartilemethod="exclusive")
    fig.for_each_xaxis(lambda x: x.update({'title': ""}) if x.title.text == 'Metric@K' else x)
    pio.write_image(fig, f"{data}_{metric}.png", format='png', scale=2, width=1400, height=650)
    fig.show()

def plot_barplot(df, data, metric, methods=None, propensities=None):
    """Plot a metric for given methods and propensities"""
    if methods:
        df = df[df['Method'].isin(methods)]
    if propensities:
        df = df[df['Propensity'].isin(propensities)]
    df = df[df['Metric'] == metric]  

    dfm = df.groupby(['Propensity', 'Method', 'Metric@K']).mean(numeric_only=True).reset_index()
    dfv = df.groupby(['Propensity', 'Method', 'Metric@K']).var(numeric_only=True).reset_index()
    dfm['var'] = dfv['Value']
   
    fig = px.bar(dfm, x="Metric@K", y="Value", color="Method", barmode="group", facet_col='Propensity', opacity=0.85, error_y="var",
     category_orders={"Method": ['bpr', 'ubpr', 'dubpr', 'relmf', 'dumf'], 'Propensity': ['Original', 'BB Item-User']})
    fig.for_each_xaxis(lambda x: x.update({'title': ""}) if x.title.text == 'Metric@K' else x)
    pio.write_image(fig, f"{data}_{metric}_bar.png", format='png', scale=2, width=1400, height=650)
    fig.show()

## Plots for New Loss functions and propensity estimation methods

In [5]:
propensities = ['original', 'bb-item-user']
_propensities = [ALL_PROPENSITIES[p] for p in propensities]
methods = ['bpr', 'ubpr', 'dubpr', 'relmf', 'dumf']      

## Plot metrics for Coat dataset

In [6]:
coat_df =load_results("logs", "coat", propensities, methods)

### DCG@K

In [7]:

plot_metric(coat_df, 'coat', 'DCG', methods=methods, propensities=_propensities)

### MAP@K

In [8]:
plot_metric(coat_df, 'coat', 'MAP', methods=methods, propensities=_propensities)


### Recall@K

In [9]:
plot_metric(coat_df, 'coat', 'Recall', methods=methods, propensities=_propensities)


## Plot metrics for Yahoo! C3 dataset

In [10]:
yahoo_df =load_results("logs", "yahoo", propensities, methods)

### DCG@K

In [11]:
plot_barplot(yahoo_df, 'yahoo', 'DCG', methods=methods, propensities=_propensities)


### MAP@K

In [12]:
plot_barplot(yahoo_df, 'yahoo', 'MAP', methods=methods, propensities=_propensities)


### Recall@K

In [13]:
plot_barplot(yahoo_df, 'yahoo', 'Recall', methods=methods, propensities=_propensities)


## Plots for NGCF results

In [14]:
propensities = ['original']
_propensities = [ALL_PROPENSITIES[p] for p in propensities]
methods = ['bpr', 'ubpr', 'ngcf_bpr', 'ngcf_ubpr']      

In [15]:
coat_df =load_results("logs", "coat", propensities, methods)

### DCG@K

In [16]:

plot_metric(coat_df, 'coat', 'DCG', methods=methods, propensities=_propensities)

### MAP@K

In [17]:

plot_metric(coat_df, 'coat', 'MAP', methods=methods, propensities=_propensities)

### Recall@K

In [18]:

plot_metric(coat_df, 'coat', 'Recall', methods=methods, propensities=_propensities)